In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
from torchsummary import summary
import numpy as np

In [4]:
# 데이터셋 인스턴스 생성

data = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")  # CSV 파일 경로
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [71]:
# 0이 나오면 안 되는 feature들: 의학적으로 0이 불가능함
data["sex"] = data["sex"].replace("female", 1)
data["sex"] = data["sex"].replace("male", 0)

data["smoker"] = data["smoker"].replace("yes", 1)
data["smoker"] = data["smoker"].replace("no", 0)

C:\Users\jinwo\AppData\Local\Temp\ipykernel_44132\3946844955.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["smoker"] = data["smoker"].replace("no", 0)


In [73]:
# 데이터와 타겟 분리
X = data.drop(['charges', 'region'], axis=1).values
y = data['charges'].values

In [75]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [77]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, 

((1070, 5), (268, 5), (1070,), (268,))

In [79]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [83]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [85]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1070, 5), (268, 5), (1070,), (268,))

In [87]:
class DiabetesEvaluationDense(nn.Module):
    def __init__(self):
        super(DiabetesEvaluationDense, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DiabetesEvaluationDense().to(device)

In [89]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [91]:
# 학습 루프
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

Epoch [10/100], Loss: 309496045.1765
Epoch [20/100], Loss: 232821098.8235
Epoch [30/100], Loss: 107948344.6471
Epoch [40/100], Loss: 48051913.4706
Epoch [50/100], Loss: 36861153.2059
Epoch [60/100], Loss: 35697646.4706
Epoch [70/100], Loss: 35688145.7941
Epoch [80/100], Loss: 34447231.0000
Epoch [90/100], Loss: 34351206.3824
Epoch [100/100], Loss: 33782192.2059


In [92]:
# 예측 및 성능 평가
model.eval()
predictions = []
true_values = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        predictions.extend(outputs.cpu().numpy())
        true_values.extend(y_batch.cpu().numpy())

# 평가 지표 계산
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(true_values, predictions)
mae = mean_absolute_error(true_values, predictions)

print(f"\n Test MSE: {mse:.4f}")
print(f" Test MAE: {mae:.4f}")


 Test MSE: 26942002.0000
 Test MAE: 3405.1484
